In [ ]:
%load_ext watermark


In [ ]:
from IPython.display import display
import numpy as np
import pandas as pd


In [ ]:
%watermark -diwmuv -iv


In [ ]:
np.random.seed(42)


## Create Dummy 3D Tensor Data


In [ ]:
nrow, ncol, nlay = 6, 9, 12
rank = 3
data = np.random.normal(size=(nrow, ncol, nlay))

assert data.shape == (nrow, ncol, nlay)

for layer in range(nlay):
    print(f"Layer {layer + 1} data:")
    display(pd.DataFrame(data[:, :, layer].reshape(nrow, ncol)))


## Create Factor Matrices


In [ ]:
A1 = np.random.rand(nrow, rank)
A2 = np.random.rand(ncol, rank)
A3 = np.random.rand(nlay, rank)

assert A1.shape == (nrow, rank)
assert A2.shape == (ncol, rank)
assert A3.shape == (nlay, rank)


## Mode 1 MTTKRP


In [ ]:
m1_matricized = data.reshape(nrow, ncol * nlay, order="F")
assert m1_matricized[0, 0] == data[0, 0, 0]
assert m1_matricized[0, 1] == data[0, 1, 0]
assert m1_matricized[1, 0] == data[1, 0, 0]
assert m1_matricized[1, 1] == data[1, 1, 0]
assert m1_matricized[0, ncol] == data[0, 0, 1]
assert m1_matricized[0, ncol + 1] == data[0, 1, 1]
assert m1_matricized[1, ncol] == data[1, 0, 1]
assert m1_matricized[1, ncol + 1] == data[1, 1, 1]


In [ ]:
kr_columns = [
    np.outer(A2[:, r], A3[:, r]).ravel()
    for r in range(rank)
]
kr_result = np.stack(kr_columns, axis=1)
assert kr_result.shape == (ncol * nlay, rank)


In [ ]:
mttkrp_result = m1_matricized @ kr_result
assert mttkrp_result.shape == (nrow, rank)


In [ ]:
pd.DataFrame(mttkrp_result)


## Mode 2 MTTKRP


In [ ]:
m2_matricized = data.transpose((1, 2, 0)).reshape(ncol, nrow * nlay, order="F")
assert m2_matricized[0, 0] == data[0, 0, 0]
assert m2_matricized[0, 1] == data[0, 0, 1]
assert m2_matricized[1, 0] == data[0, 1, 0]
assert m2_matricized[1, 1] == data[0, 1, 1]
assert m2_matricized[0, nlay] == data[1, 0, 0]
assert m2_matricized[0, nlay + 1] == data[1, 0, 1]
assert m2_matricized[1, nlay] == data[1, 1, 0]
assert m2_matricized[1, nlay + 1] == data[1, 1, 1]


In [ ]:
kr_columns = [
    np.outer(A1[:, r], A3[:, r]).ravel()
    for r in range(rank)
]
kr_result = np.stack(kr_columns, axis=1)
assert kr_result.shape == (nrow * nlay, rank)


In [ ]:
mttkrp_result = m2_matricized @ kr_result
assert mttkrp_result.shape == (ncol, rank)


In [ ]:
pd.DataFrame(mttkrp_result)


## Mode 3 MTTKRP


In [ ]:
m3_matricized = data.transpose((2, 0, 1)).reshape(nlay, nrow * ncol, order="F")
assert m3_matricized[0, 0] == data[0, 0, 0]
assert m3_matricized[0, 1] == data[1, 0, 0]
assert m3_matricized[1, 0] == data[0, 0, 1]
assert m3_matricized[1, 1] == data[1, 0, 1]
assert m3_matricized[0, nrow] == data[0, 1, 0]
assert m3_matricized[0, nrow + 1] == data[1, 1, 0]
assert m3_matricized[1, nrow] == data[0, 1, 1]
assert m3_matricized[1, nrow + 1] == data[1, 1, 1]


In [ ]:
kr_columns = [
    np.outer(A1[:, r], A2[:, r]).ravel()
    for r in range(rank)
]
kr_result = np.stack(kr_columns, axis=1)
assert kr_result.shape == (ncol * nrow, rank)


In [ ]:
mttkrp_result = m3_matricized @ kr_result
assert mttkrp_result.shape == (nlay, rank)


In [ ]:
pd.DataFrame(mttkrp_result)
